##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Estimator を使ったマルチワーカートレーニング

<table class="tfo-notebook-buttons" align="left">
  <td> <img src="https://www.tensorflow.org/images/tf_logo_32px.png"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/distribute/multi_worker_with_estimator.ipynb">TensorFlow.org で表示</a> </td>
  <td> <img src="https://www.tensorflow.org/images/colab_logo_32px.png"><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/distribute/multi_worker_with_estimator.ipynb">Google Colab で実行</a> </td>
  <td> <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/distribute/multi_worker_with_estimator.ipynb">GitHub でソースを表示</a> </td>
  <td> <img src="https://www.tensorflow.org/images/download_logo_32px.png"><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/distribute/multi_worker_with_estimator.ipynb">ノートブックをダウンロード</a> </td>
</table>

> Warning: Estimators are not recommended for new code.  Estimators run `v1.Session`-style code which is more difficult to write correctly, and can behave unexpectedly, especially when combined with TF 2 code. Estimators do fall under [compatibility guarantees](https://tensorflow.org/guide/versions), but will receive no fixes other than security vulnerabilities. See the [migration guide](https://tensorflow.org/guide/migrate) for details.

## 概要

注意: `tf.distribute` API で Estimator を使用することもできますが、`tf.distribute`で Keras を使用することを推奨します。[Keras を使ったマルチワーカートレーニング](multi_worker_with_keras.ipynb)をご覧ください。`tf.distribute.Strategy` を使った Estimator のトレーニングのサポートは制限されています。

このチュートリアルでは、`tf.estimator`を使った分散型マルチワーカートレーニングに `tf.distribute.Strategy` を使用する方法を実演します。`tf.estimator` を使って独自のコードを記述し、高性能のマシン一台以上のスケーリングを希望する場合は、このチュートリアルをご利用ください。

始める前に、[分散ストラテジー](../../guide/distributed_training.ipynb)ガイドをお読みください。[マルチ GPU トレーニングのチュートリアル](./keras.ipynb)も関連しています。このチュートリアルでは同じモデルが使用されています。


## セットアップ

最初に、TensorFlow と必要なインポートをセットアップします。

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

import os, json

注意: TF2.4 以降、マルチワーカーミラーリングストラテジーは、eager を有効にして実行すると (デフォルト)、Estimator で失敗します。TF2.4 のエラーは `TypeError: cannot pickle '_thread.lock' object` です。詳細については、[課題 #46556](https://github.com/tensorflow/tensorflow/issues/46556) を参照してください。回避策は、eager の実行を無効にすることです。

In [ ]:
tf.compat.v1.disable_eager_execution()

## 入力関数

このチュートリアルでは、[TensorFlow Datasets](https://www.tensorflow.org/datasets) の MNIST データセットを使用しています。このコードは[マルチ GPU トレーニングのチュートリアル](./keras.ipynb)のコードに似ていますが、大きな違いが 1 つあります。マルチワーカートレーニングに Estimator を使用する際は、モデルのコンバージェンスを可能にするように、ワーカーの数でデータセットをシャーディングする必要があります。入力データは、ワーカーインデックスでシャーディングされるため、各ワーカーは、データセットの各 `1/num_workers` の部分を処理します。

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

def input_fn(mode, input_context=None):
  datasets, info = tfds.load(name='mnist',
                                with_info=True,
                                as_supervised=True)
  mnist_dataset = (datasets['train'] if mode == tf.estimator.ModeKeys.TRAIN else
                   datasets['test'])

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

  if input_context:
    mnist_dataset = mnist_dataset.shard(input_context.num_input_pipelines,
                                        input_context.input_pipeline_id)
  return mnist_dataset.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

コンバージェンスを達成するためのもう 1 つの合理的なアプローチとして、各ワーカーで異なるシードを使ってデータベースをシャッフルする方法があります。

## マルチワーカー構成

このチュートリアルの主な違いの 1 つに ([マルチ GPU トレーニングのチュートリアル](./keras.ipynb) と比較)、マルチワーカーのセットアップがあります。`TF_CONFIG` 環境変数は、クラスタの一部である各ワーカーにクラスタ構成を指定する標準的な方法です。

`TF_CONFIG` には、`cluster` と `task` の 2 つのコンポーネントがあります。`cluster` は、クラスタのワーカーとパラメータサーバーを含むクラスタ全体に関する情報を提供するのに対し、`task` は、現在のタスクに関する情報を提供します。最初のコンポーネント `cluster` は、クラスタ内のすべてのワーカーとパラメータサーバーで同一であり、2 つ目のコンポーネント `task` は、各ワーカーとパラメータサーバー間で異なり、それぞれに `type` と `index` を指定します。この例では、タスクの `type` は `worker` で、タスクの `index` は `0` です。

例示を目的として、このチュートリアルでは、`localhost` 上に 2 つのワーカーを持つ `TF_CONFIG` の設定方法を示しています。実際には、外部 IP アドレスとポートに複数のワーカーを作成し、各ワーカーに適切に `TF_CONFIG` を設定します (タスクの `index` を変更します)。

警告: *Colab で次のコードを実行しないでください。*TensorFlow のランタイムは、指定された IP アドレスとポートに gRPC サーバーを作成しようとしますが、失敗する可能性があります。1 台のマシンで複数のワーカーをテスト実行する方法の例については、このチュートリアルの [keras バージョン](multi_worker_with_keras.ipynb)を参照してください。

```
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["localhost:12345", "localhost:23456"]
    },
    'task': {'type': 'worker', 'index': 0}
})
```


## モデルを定義する

トレーニング用にレイヤー、オプティマイザ、および損失関数を記述します。このチュートリアルでは、[マルチ GPU トレーニングのチュートリアル](./keras.ipynb)と同様に、Keras レイヤーを使ったモデルを定義しています。

In [ ]:
LEARNING_RATE = 1e-4
def model_fn(features, labels, mode):
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  logits = model(features, training=False)

  if mode == tf.estimator.ModeKeys.PREDICT:
    predictions = {'logits': logits}
    return tf.estimator.EstimatorSpec(labels=labels, predictions=predictions)

  optimizer = tf.compat.v1.train.GradientDescentOptimizer(
      learning_rate=LEARNING_RATE)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(labels, logits)
  loss = tf.reduce_sum(loss) * (1. / BATCH_SIZE)
  if mode == tf.estimator.ModeKeys.EVAL:
    return tf.estimator.EstimatorSpec(mode, loss=loss)

  return tf.estimator.EstimatorSpec(
      mode=mode,
      loss=loss,
      train_op=optimizer.minimize(
          loss, tf.compat.v1.train.get_or_create_global_step()))

注意: この例の学習速度は固定されていますが、一般的に、グローバルバッチサイズに基づいて学習速度を調整する必要があります。

## MultiWorkerMirroredStrategy

モデルをトレーニングするために、`tf.distribute.experimental.MultiWorkerMirroredStrategy` のインスタンスを使用します。`MultiWorkerMirroredStrategy` は、すべてのワーカーの各デバイスにあるモデルのレイヤーにすべての変数のコピーを作成します。集合通信に使用する TensorFlow 演算子 `CollectiveOps` を使用して勾配を集め、変数の同期を維持します。このストラテジーの詳細は、[`tf.distribute.Strategy` ガイド](../../guide/distributed_training.ipynb)で説明されています。

In [ ]:
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

## モデルをトレーニングして評価する

次に、分散ストラテジーを Estimator の `RunConfig` に指定し、`tf.estimator.train_and_evaluate` を呼び出してトレーニングと評価を行います。このチュートリアルでは、`train_distribute` 経由でストラテジーを指定してトレーニングのみを分散しています。`eval_distribute` を使って評価を分散することもできます。

In [ ]:
config = tf.estimator.RunConfig(train_distribute=strategy)

classifier = tf.estimator.Estimator(
    model_fn=model_fn, model_dir='/tmp/multiworker', config=config)
tf.estimator.train_and_evaluate(
    classifier,
    train_spec=tf.estimator.TrainSpec(input_fn=input_fn),
    eval_spec=tf.estimator.EvalSpec(input_fn=input_fn)
)

## トレーニングのパフォーマンスを最適化する

`tf.distribute.Strategy` により、モデルとマルチワーカー対応 Estimator の準備が整いました。次のテクニックに従って、マルチワーカートレーニングのパフォーマンスを最適化することができます。

- *バッチサイズの増加:* ここで指定されるバッチサイズは、GPU 単位のサイズです。一般的に、GPU メモリに収まる最大バッチサイズの指定が推奨されます。

- *変数のキャスト:* 可能であれば、`tf.float` に変数をキャストしてください。公式の ResNet モデルには、どのようにしてこれを行うかの[例](https://github.com/tensorflow/models/blob/8367cf6dabe11adf7628541706b660821f397dce/official/resnet/resnet_model.py#L466)が示されています。

- *集合通信の使用:* `MultiWorkerMirroredStrategy` は、複数の[集合通信実装](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/distribute/cross_device_ops.py)を提供しています。

    - `RING` は、クロスホスト通信レイヤーとして、gRPC を使用したリング状の集合体を実装します。
    - `NCCL` は、[Nvidia の NCCL](https://developer.nvidia.com/nccl) を使用して集合体を実装します。
    - `AUTO` は、選択をランタイムに持ち越します。

    最適な集合体実装の選択肢は、GPU 数と種類、および、クラスタ内の相互接続ネットワークにより異なります。自動選択をオーバーライドするには、`MultiWorkerMirroredStrategy` コンストラクタの `communication` パラメータに、 `communication=tf.distribute.experimental.CollectiveCommunication.NCCL` のように有効な値を指定します。

ガイドの[パフォーマンスのセクション](../../guide/function.ipynb)に目を通し、独自の TensorFlow モデルのパフォーマンス最適化に使用できるほかのストラテジーや[ツール](../../guide/profiler.md)についてさらに詳しく学習しましょう。


## その他のコード例

1. Kubernetes テンプレートを使った tensorflow/ecosystem でマルチワーカートレーニングを行うための[エンドツーエンドの例](https://github.com/tensorflow/ecosystem/tree/master/distribution_strategy)。この例は最初に Keras モデルを使用し、それを `tf.keras.estimator.model_to_estimator` API を使って Estimator に変換します。
2. [公式モデル](https://github.com/tensorflow/models/tree/master/official)。この多くは、複数の分散ストラテジーで実行するように構成できます。
